# 모델 개요
- **기본 가설**: 깊이 분리 가능한 컨볼루션 레이어를 사용하여 채널 간 상관관계와 공간 상관관계를 완전히 분리할 수 있는지 실험.
- **아키텍처 이름**: "Extreme Inception"의 줄임말로 Xception이라 명명.
  
- **구조**
  - **36개의 컨볼루션 레이어**: 네트워크의 특징 추출을 담당.
  - **로지스틱 회귀 레이어**: 컨볼루션 베이스 후에 사용.
  - **선택적 완전 연결 레이어**: 실험적 평가에서 추가 가능.
  - **14개의 모듈**: 첫 번째와 마지막 모듈을 제외하고 모두 선형 잔차 연결(Linear Residual Connections)을 가짐.
  
- **장점**
  - **쉽게 정의 및 수정 가능**: Keras 또는 TensorFlow-Slim을 사용하여 30~40 줄의 코드로 아키텍처 정의 가능.
  - **복잡도 감소**: VGG-16과 비교하여 정의와 수정이 쉬우며, Inception V2/V3보다 간단.
- **오픈 소스 구현**: Keras 및 TensorFlow에서 MIT 라이선스로 제공됨.


# Inception V3와의 차이점

## 설계 차이점
1. **인셉션 모듈 vs. Depthwise Separable Convolutions**
   - **Inception V3**:
     - 인셉션 모듈을 사용하여 여러 종류의 필터(1x1, 3x3, 5x5)와 풀링 계층을 병렬로 적용합니다.
     - 각 모듈은 1x1 합성곱으로 교차-채널 상관관계를 매핑하고, 그 다음 단계에서 공간적 상관관계를 매핑합니다.
   - **Xception**:
     - Depthwise Separable Convolutions 사용: 한 채널씩 독립적으로 공간적 합성곱을 수행한 후 1x1 합성곱으로 채널을 다시 매핑합니다.
     - Xception은 이러한 깊이별 분리 합성곱을 연속적으로 쌓아 올린 구조로 이루어져 있습니다.
   
2. **연결 방식**
   - **Inception V3**: 각 인셉션 모듈을 조합하여 네트워크를 구성합니다.
   - **Xception**: 36개의 Depthwise Separable Convolutions 레이어를 선형으로 쌓고, 각 모듈 주변에 선형 잔차 연결(Residual connection)을 사용합니다.
   
3. **비선형성**
   - **Inception V3**: 비선형 활성화 함수(ReLU)를 각 1x1 합성곱과 그 후의 합성곱 후에 추가합니다.
   - **Xception**: Depthwise Separable Convolutions에서는 활성화 함수를 사용하지 않는 경우가 많습니다.



## 장단점
- **Inception V3**
  - **장점**:
    - 다양한 크기의 필터와 병렬 계층 덕분에 매우 효율적으로 다양한 크기의 특징을 추출할 수 있습니다.
  - **단점**:
    - 비교적 복잡한 구조로 인해 모델 정의와 수정이 어렵습니다.
- **Xception**
  - **장점**:
    - Depthwise Separable Convolutions로 인해 파라미터 수가 줄어들고 계산 효율성이 높아집니다.
    - 선형 구조와 Residual 연결을 사용해 모델 정의와 수정을 쉽게 할 수 있습니다.
  - **단점**:
    - 강력한 디커플링 가정이 항상 최적의 성능을 보장하지 않을 수 있습니다.

## 성능 비교
- **파라미터 수**: 두 모델 모두 비슷한 수의 파라미터를 가지고 있습니다.
- **ImageNet 데이터셋 성능**:
  - Xception은 ImageNet 데이터셋에서 Inception V3와 유사한 성능을 보이거나 약간 더 우수한 성능을 보입니다.
- **대규모 이미지 분류 성능 (JFT 데이터셋)**:
  - Xception은 Inception V3보다 큰 마진으로 뛰어난 성능을 보입니다. 이는 350만개의 이미지와 17,000개의 클래스로 구성된 대규모 데이터셋에서 확인된 사항입니다.

## 결론
- **Inception V3**는 다양한 크기의 필터를 병렬로 적용하는 고도의 복잡성을 자랑하며, 충분히 최적화된 모델입니다.
- **Xception**은 동일한 파라미터 수로 높은 효율성을 자랑하며, 특히 대규모 데이터셋에서 뛰어난 성능을 보입니다. Depthwise Separable Convolutions을 통해 더 효율적으로 계산할 수 있게 합니다.


# Data Prepairing

In [107]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

In [108]:
transform = transforms.Compose(
    [
        # transforms.Resize(224),
        # transforms.RandomCrop((224, 224), padding=4),
        transforms.RandomCrop((32, 32), padding=4),
        transforms.RandomVerticalFlip(0.5),
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ]
)

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


# Modeling

## Depthwise Separable Convolutions

Conv($W$, $y$)$_{(i,j)} = \sum_{k,l,m}^{K, L, M} W_{(k,l,m)} \cdot y_{(i+k,j+l,m)}$

PointwiseConv($W$, $y$)$_{(i,j)} = \sum_{m}^{M} W_{m} \cdot y_{(i,j,m)}$

DepthwiseConv($W$, $y$)$_{(i,j)} = \sum_{k,l}^{K, L} W_{(k,l)} \odot y_{(i+k,j+l)}$

SepConv($W_p$, $W_d$, $y$)$_{(i,j)}$ = $\text{PointwiseConv}_{(i, j)}(W_{p}, \text{DepthwiseConv}_{(i, j)}(W_{d}, y))$



### Depthwise Convolution

![](https://raw.githubusercontent.com/seungjunlee96/Depthwise-Separable-Convolution_Pytorch/master/images/depthwise.png)
> Depthwise Convolution Diagram  
https://github.com/seungjunlee96/Depthwise-Separable-Convolution_Pytorch


In [109]:
class DepthwiseConv(nn.Module):
  def __init__(self, in_channels):
    super(DepthwiseConv, self).__init__()
    self.conv = nn.Conv2d(in_channels, in_channels,
                          kernel_size=3, padding=1, groups=in_channels,
                          bias=False)

  def forward(self, x):
    out = self.conv(x)
    return out

### Pointwise Convolution

![](https://raw.githubusercontent.com/seungjunlee96/Depthwise-Separable-Convolution_Pytorch/master/images/pointwise.png)

> Pointwise Convolution Diagram  
https://github.com/seungjunlee96/Depthwise-Separable-Convolution_Pytorch

In [110]:
class PointwiseConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(PointwiseConv, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)

  def forward(self, x):
    out = self.conv(x)
    return out

### Depthwise Separable Convolution

In [111]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = DepthwiseConv(in_channels)
        self.pointwise = PointwiseConv(in_channels, out_channels)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

## Entry Flow

In [112]:
class EntryFlow(nn.Module):
    class ConvBlock(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(EntryFlow.ConvBlock, self).__init__()

            self.conv_block = nn.Sequential(
                DepthwiseSeparableConv(in_channels, out_channels),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(True),
                DepthwiseSeparableConv(out_channels, out_channels),
                nn.BatchNorm2d(out_channels),
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
            )

            self.conv_shortcut = nn.Conv2d(in_channels, out_channels,
                                           kernel_size=1, stride=2, padding=0)

        def forward(self, x):
            out = self.conv_block(x)
            shortcut = self.conv_shortcut(x)
            out = out + shortcut
            return out

    def __init__(self):
        super(EntryFlow, self).__init__()

        self.boostrap = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )

        self.conv_blocks = nn.Sequential(
            self.ConvBlock(64, 128),
            self.ConvBlock(128, 256),
            self.ConvBlock(256, 728)
        )

    def forward(self, x):
        out = self.boostrap(x)
        out = self.conv_blocks(out)
        return out

## Middle Flow

In [113]:
class MiddleFlow(nn.Module):
    class ResidualBlock(nn.Module):
        def __init__(self, in_channels):
            super(MiddleFlow.ResidualBlock, self).__init__()

            self.residual_block = nn.Sequential(
                nn.ReLU(True),
                DepthwiseSeparableConv(in_channels, in_channels),
                nn.BatchNorm2d(in_channels),
                nn.ReLU(True),
                DepthwiseSeparableConv(in_channels, in_channels),
                nn.BatchNorm2d(in_channels),
                nn.ReLU(True),
                DepthwiseSeparableConv(in_channels, in_channels),
                nn.BatchNorm2d(in_channels)
            )

        def forward(self, x):
            out = self.residual_block(x)
            out = out + x
            return out

    def __init__(self):
        super(MiddleFlow, self).__init__()

        self.residual_blocks = nn.Sequential(
            *[self.ResidualBlock(728) for _ in range(8)]
        )

    def forward(self, x):
        out = self.residual_blocks(x)
        return out


## Exit Flow

In [114]:
class ExitFlow(nn.Module):
    def __init__(self):
        super(ExitFlow, self).__init__()

        self.conv_block1 = nn.Sequential(
            nn.ReLU(True),
            DepthwiseSeparableConv(728, 728),
            nn.BatchNorm2d(728),
            nn.ReLU(True),
            DepthwiseSeparableConv(728, 1024),
            nn.BatchNorm2d(1024),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv_shortcut = nn.Conv2d(728, 1024,
                                       kernel_size=1, stride=2, padding=0)

        self.conv_block2 = nn.Sequential(
            DepthwiseSeparableConv(1024, 1536),
            nn.BatchNorm2d(1536),
            nn.ReLU(True),
            DepthwiseSeparableConv(1536, 2048),
            nn.BatchNorm2d(2048),
            nn.ReLU(True)
        )

        self.linear = nn.Linear(2048, 10)

    def forward(self, x):
        out = self.conv_block1(x)
        shortcut = self.conv_shortcut(x)
        out = out + shortcut
        out = self.conv_block2(out)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

## Xception

In [115]:
class Xception(nn.Module):
    def __init__(self):
        super(Xception, self).__init__()

        self.entry_flow = EntryFlow()
        self.middle_flow = MiddleFlow()
        self.exit_flow = ExitFlow()

    def forward(self, x):
        out = self.entry_flow(x)
        out = self.middle_flow(out)
        out = self.exit_flow(out)
        return out

# Training

In [116]:
import tqdm
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Xception()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
for epoch in range(num_epochs):
    iterator = tqdm.tqdm(train_loader)
    model.train()
    for images, labels in iterator:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        iterator.set_description(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/10], Loss: 1.0393: 100%|██████████| 3125/3125 [01:49<00:00, 28.45it/s]


# Testing

In [117]:
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    iterator = tqdm.tqdm(test_loader)
    for images, labels in iterator:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'\nAccuracy of the model on the test images: {100 * correct / total:.2f}%')


100%|██████████| 625/625 [00:08<00:00, 72.03it/s]


Accuracy of the model on the test images: 68.80%
